This program takes in the reference_location csv and returns one result with highest confidence (according to OSM). This program uses Nominatim, a tool to search OSM data by name and address and generate co-ordinates. 
Additional information can be added from query results by adding additional fields after 'g = geocoder.osm(item)'

Inelegant error handling of query 'Antarctica'. 

References:
- https://nominatim.openstreetmap.org/ui/search.html
- https://geocoder.readthedocs.io/api.html#error-handling

Information oin different feature types can be found here:
- https://github.com/osm-search/Nominatim/blob/80df4d3b560f5b1fd550dcf8cdc09a992b69fee0/settings/partitionedtags.def
- https://nominatim.org/release-docs/latest/api/Lookup/ 





In [3]:
"""
Write a program that takes in a list of placenames (country, capital etc) and queries the osm API to return latitudes and longitudes for each of them.
Print these results to a csv file. Use geopy and geocoder to do the lookup. 
"""

import requests
import json
import sys
import pandas as pd
import geocoder
import geopy

df = pd.read_csv('/Users/fiannualamorgan/Documents/GitHub/geolocation-tools-workshop/Data/Aust_Colonies.csv')

#build query string for each row in df and store in list 
query_list = []

# add items from first column of df to query_list except the heading
for item in df.iloc[:,0]:
    query_list.append(item)

#build query string for each row in df for geocoder.osm and store results in dictionary with key as placename and value as lat,lon
geocoder_results = {}

for item in query_list:
    print(f'constructing query string for {item}')
    if item == 'Antarctica':
        geocoder_results[item] = 'None'
    else:
        g = geocoder.osm(item)
        if g.ok:
            geocoder_results[item] = g.latlng
        else:
            geocoder_results[item] = 'None'
            print(f'{item} not found')

constructing query string for Melbourne
constructing query string for Brisbane
constructing query string for Perth
constructing query string for Darwin
constructing query string for Alice Springs
constructing query string for Ballarat
constructing query string for Canberra


In [6]:
import folium
from folium import plugins

aust_map= folium.Map(location=[-37.2094444, 144.2094444], zoom_start=5)
placenames = plugins.MarkerCluster().add_to(aust_map)

#use geocoder_results to add markers to map for each placename in geocoder_results dictionary
for key, value in geocoder_results.items():
    if value != 'None':
        lat, lon = value
        placename = key
        folium.Marker([lat, lon], popup=placename).add_to(placenames)
    else:
        print(f'{key} not found')

aust_map





In [7]:
#save to csv file
geocoder_results_df = pd.DataFrame(list(geocoder_results.items()), columns=['LocationName', 'Latitude, Longitude'])
geocoder_results_df.to_csv('/Users/fiannualamorgan/Documents/GitHub/geolocation-tools-workshop/geocoder_results.csv')